In [1]:
import xmltodict

with open('ds_od_xml.xml') as f:
    ds_d = xmltodict.parse(f.read())

In [2]:
from ds_types import Establishment, Inspection, Infraction, get_establishment
from typing import List

establishments = []
for i, d in enumerate(ds_d['DINESAFE_DATA']['ESTABLISHMENT']):
    try:
        establishments.append(get_establishment(d))
    except Exception as e:
        print(e)
        print(d)
        break